ANNOTATIONS WITH LABELME

In [9]:
!labelme

2025-02-04 03:25:45,576 [INFO   ] __init__:get_config:67- Loading config file from: C:\Users\User\.labelmerc
2025-02-04 04:04:18,747 [ERROR  ] label_file:load_image_file:50- Failed opening image file: G:\rickshaw-car-motorcycle\dataset\022ec329-6675-44d8-922d-70af48eba824.jpg
2025-02-04 04:04:40,353 [ERROR  ] label_file:load_image_file:50- Failed opening image file: G:\rickshaw-car-motorcycle\dataset\022ec329-6675-44d8-922d-70af48eba824.jpg
2025-02-04 04:06:18,590 [ERROR  ] label_file:load_image_file:50- Failed opening image file: G:\rickshaw-car-motorcycle\dataset\022ec329-6675-44d8-922d-70af48eba824.jpg


AUGMENTATION WITH ALBUMENTATIONS

In [10]:
image_folder = "G:/rickshaw-car-motorcycle/dataset"
annotations = "G:/rickshaw-car-motorcycle/dataset_annotations.json"
output_image_folder = "G:/rickshaw-car-motorcycle/aug_dataset"
output_annotations = "G:/rickshaw-car-motorcycle/aug_dataset_annotations.json"

import json
import albumentations as A
import os
import cv2

os.makedirs(output_image_folder, exist_ok=True)

# Puran coco annotation load kore nilam
with open(annotations, "r") as f:
    d = json.load(f)
    
# ki ki and kemon kemon transformation hobe set kore dilam
# bounding box keo handle korte hobe bole dilam, format o diye dilam
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.RandomRotate90(p=0.5),
    A.Blur(p=0.5),
    A.Resize(224, 224),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_id']))  

# notun image and annotation er list banailam and oitar moddhe puran data dhukay rakhlam
new_images = d["images"].copy()
new_annotations = d["annotations"].copy()

# notun image and annotations er id number koto hobe eita bair kore nilam. max er theke 1 beshi kore barte thakbe
image_id = max(img["id"] for img in new_images) + 1
annotation_id = max(ann["id"] for ann in new_annotations) + 1  


# ekta ekta kore image niya ashlam
for img_info in d["images"]:
    image_path = os.path.join(image_folder, img_info["file_name"])
    image = cv2.imread(image_path)
    
    # image e jhamela thakle oita bad diya move on
    if image is None:
        print(f"{image_path} not found")
        continue
    
    # oi image er annotation gula niye ashlam
    image_annotations = [ann for ann in d["annotations"] if ann["image_id"] == img_info["id"]]
    
    # annotation gula theke bounding box and category id bair koira nilam
    bboxes = [ann["bbox"] for ann in image_annotations]
    category_ids = [ann["category_id"] for ann in image_annotations]
    
    for i in range(3):
        # dilam augmentation chalaiya
        augmented = transform(image=image, bboxes=bboxes, category_id=category_ids)
    
        # augmented image er ekta notun nam dilam ar koi save hobe seita set koira dilam
        base_name = os.path.basename(img_info["file_name"])
        new_file_name = f"{os.path.splitext(base_name)[0]}_aug{i}.jpg"
        new_image_path = os.path.join(output_image_folder, new_file_name)
    
        # augmented image save koira dilam
        cv2.imwrite(new_image_path, augmented["image"])
    
        # augmented image take notun image list e dhukay dilam dorkari info shoho
        new_images.append({
            "id": image_id,
            "file_name": new_file_name,
            "height": 224,
            "width": 224
        })
    
        # annotation gulakeo save koira nilam
        # annotation er number increease koira dilam
        # image er number o increase koira dilam
        for bbox, category_id in zip(augmented["bboxes"], category_ids):
            new_annotations.append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": bbox
            })
            annotation_id += 1
        image_id += 1
    
    
    
# notun ekta coco format er dictionary banailam
# augment howa image ar annotations shob eksathe eikhane joma korlam    
new_coco_data = {
    "images": new_images,
    "annotations": new_annotations,
    "categories": d["categories"]
}        

# purata ke json e dump korlam. hoiya gelo notun dataset and annotations
with open(output_annotations, "w") as f:
    json.dump(new_coco_data, f)
    
    
    

split train and validation

In [ ]:
folder = "G:/rickshaw-car-motorcycle/aug_dataset"
annotations = "G:/rickshaw-car-motorcycle/aug_dataset/aug_dataset_annotations.json"

import random
import shutil
import json
import os

# json fie load kore nilam
with open(annotations, "r") as f:
    coco_data = json.load(f)

# random shuffle kore dilam
# 0 theke 500 porjonto train e, baki valid e    
random.shuffle(coco_data["images"])
train_images = coco_data["images"][:500]
val_images = coco_data["images"][500:]

# train and valid er id gula alada kore nilam
train_ids = {img["id"] for img in train_images}
val_ids = {img["id"] for img in val_images}

# train and valid er annotation gula alada kore nilam
train_annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] in train_ids]
val_annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] in val_ids]    

# train and valid er json file banailam
train_coco = {"images": train_images, "annotations": train_annotations, "categories": coco_data["categories"]}
val_coco = {"images": val_images, "annotations": val_annotations, "categories": coco_data["categories"]}

# train and valid er folder banailam
train_folder = "G:/rickshaw-car-motorcycle/aug_dataset/train"
valid_folder = "G:/rickshaw-car-motorcycle/aug_dataset/valid"
# sure hoye nilam folder asholei ase to!
os.makedirs(train_folder, exist_ok=True)
os.makedirs(valid_folder, exist_ok=True)

# json file er path save korlam
train_json_path = os.path.join(train_folder, "train_annotations.json")
val_json_path = os.path.join(valid_folder, "valid_annotations.json")

# json file e dump kore dilam 
with open(train_json_path, "w") as f:
    json.dump(train_coco, f)

with open(val_json_path, "w") as f:
    json.dump(val_coco, f)
    
    
# image gulao copy kore relevent folder e pathay dilam
for img in train_images:
    file_name = os.path.basename(img["file_name"])  
    shutil.copy(os.path.join(folder, file_name), train_folder)

for img in val_images:
    file_name = os.path.basename(img["file_name"])  
    shutil.copy(os.path.join(folder, file_name), valid_folder)   

YOLO format conversion

In [28]:
import os
import json
from pycocotools.coco import COCO
import shutil

# shob dorkari path gula niye nilam
train_annotation_file = 'G:/rickshaw-car-motorcycle/aug_dataset/train/train_annotations.json'
valid_annotation_file = 'G:/rickshaw-car-motorcycle/aug_dataset/valid/valid_annotations.json'
train_image_dir = 'G:/rickshaw-car-motorcycle/aug_dataset/train' 
valid_image_dir = 'G:/rickshaw-car-motorcycle/aug_dataset/valid'  


output_train_images_dir = 'G:/rickshaw-car-motorcycle/aug_dataset/yolo-train/images'
output_train_labels_dir = 'G:/rickshaw-car-motorcycle/aug_dataset/yolo-train/labels'
output_valid_images_dir = 'G:/rickshaw-car-motorcycle/aug_dataset/yolo-valid/images'
output_valid_labels_dir = 'G:/rickshaw-car-motorcycle/aug_dataset/yolo-valid/labels'

# folder gula asholei ase to !
os.makedirs(output_train_images_dir, exist_ok=True)
os.makedirs(output_train_labels_dir, exist_ok=True)
os.makedirs(output_valid_images_dir, exist_ok=True)
os.makedirs(output_valid_labels_dir, exist_ok=True)

# conversion function 
def convert_coco_to_yolo(annotation_file, image_dir, output_images_dir, output_labels_dir):
   
    coco = COCO(annotation_file)
    
    # category gula load kore nilam
    # class name and id ke ekta dictionary te raikha dilam
    categories = coco.loadCats(coco.getCatIds())
    class_names = [category['name'] for category in categories]
    class_dict = {category['id']: idx for idx, category in enumerate(categories)}

    # image gula load koira nilam
    image_ids = coco.getImgIds()
    for image_id in image_ids:
        image_info = coco.loadImgs(image_id)[0]
        image_filename = image_info['file_name']

        # image er path banailam
        image_path = os.path.join(image_dir, os.path.basename(image_filename))

        # image na thakle skip kore chole jabe and warning show kore dibe
        if not os.path.exists(image_path):
            print(f"Warning: {image_path} does not exist. Skipping this file.")
            continue

        
        basename = os.path.basename(image_filename)

        # image copy koira dilam
        shutil.copy(image_path, os.path.join(output_images_dir, basename))

        # annotation gula load koira nilam
        annotation_ids = coco.getAnnIds(imgIds=image_id)
        annotations = coco.loadAnns(annotation_ids)

        # label file er path banailam
        label_filename = basename.replace('.jpg', '.txt')
        label_filepath = os.path.join(output_labels_dir, label_filename)

        # label file e data write koira dilam
        with open(label_filepath, 'w') as label_file:
            for annotation in annotations:
                
                # bounding box er center, width, height calculate koira dilam
                bbox = annotation['bbox']
                x_center = (bbox[0] + bbox[2] / 2) / image_info['width']
                y_center = (bbox[1] + bbox[3] / 2) / image_info['height']
                width = bbox[2] / image_info['width']
                height = bbox[3] / image_info['height']

                # class id and bounding box er data write koira dilam
                category_id = annotation['category_id']
                if category_id in class_dict:
                    label_file.write(f"{class_dict[category_id]} {x_center} {y_center} {width} {height}\n")
                else:
                    print(f"Warning: Category ID {category_id} not found in class_dict")

    print(f"COCO to YOLO conversion complete for {annotation_file}.")


convert_coco_to_yolo(train_annotation_file, train_image_dir, output_train_images_dir, output_train_labels_dir)
convert_coco_to_yolo(valid_annotation_file, valid_image_dir, output_valid_images_dir, output_valid_labels_dir)

print("COCO to YOLO conversion complete for both train and valid datasets.")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
COCO to YOLO conversion complete for G:/rickshaw-car-motorcycle/aug_dataset/train/train_annotations.json.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
COCO to YOLO conversion complete for G:/rickshaw-car-motorcycle/aug_dataset/valid/valid_annotations.json.
COCO to YOLO conversion complete for both train and valid datasets.


Train Model. Kaggle notebook is available in the folder

In [33]:
from ultralytics import YOLO

model_path = 'yolov8s.pt'
data_path = 'G:/rickshaw-car-motorcycle/data.yaml'

model = YOLO(model_path)


lr0 = 0.001 
lrf = 0.1
batch = 16 
epochs = 2
optimizer = 'Adam'  


model.train(data=data_path, epochs=epochs, imgsz=640, batch=batch, lr0=lr0, lrf=lrf, optimizer=optimizer)

New https://pypi.org/project/ultralytics/8.3.71 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.34  Python-3.10.14 torch-2.5.1 CPU (Intel Core(TM) i5-8265U 1.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=G:/rickshaw-car-motorcycle/data.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, 

train: Scanning G:\rickshaw-car-motorcycle\yolo-dataset\train\labels... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:01<00:00, 393.17it/s]

train: New cache created: G:\rickshaw-car-motorcycle\yolo-dataset\train\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



c:\Users\User\miniconda3\envs\acienv\lib\site-packages\ultralytics\data\augment.py:1850: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0),
val: Scanning G:\rickshaw-car-motorcycle\yolo-dataset\valid\labels... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 394.25it/s]


val: New cache created: G:\rickshaw-car-motorcycle\yolo-dataset\valid\labels.cache
Plotting labels to runs\detect\train3\labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train3
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2         0G      1.671      2.258      1.565         25        640: 100%|██████████| 32/32 [14:43<00:00, 27.61s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [01:08<00:00, 17.07s/it]

                   all        100        300      0.452      0.238      0.225      0.118



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/2         0G      1.458       1.88      1.479         70        640:   3%|▎         | 1/32 [01:16<39:37, 76.69s/it]


KeyboardInterrupt: 